In [1]:
%load_ext autoreload
%autoreload 2

In [1]:
#Install the forked cell-load
#!pip install --upgrade git+https://github.com/rkita/cell-load.git@scplode-integration
#%pip install -q git+https://github.com/rkita/cell-load.git@scplode-integration


In [2]:
#Download Competition Support Set

/Users/ryo/miniconda3/envs/colab-mimic/bin/python
['/Users/ryo/miniconda3/envs/colab-mimic/lib/python311.zip', '/Users/ryo/miniconda3/envs/colab-mimic/lib/python3.11', '/Users/ryo/miniconda3/envs/colab-mimic/lib/python3.11/lib-dynload', '', '/Users/ryo/miniconda3/envs/colab-mimic/lib/python3.11/site-packages', '/Users/ryo/Desktop/external_repos/cell-load/src']
/Users/ryo/Desktop/external_repos/cell-load/src/cell_load/__init__.py


In [2]:
from cell_load.data_modules import PerturbationDataModule
import toml

In [13]:
#Make sure the version of PerturbationDataModule  is the right one
import inspect
sig = inspect.signature(PerturbationDataModule.__init__)
assert 'use_scplode' in sig.parameters

In [14]:
config = {
    "datasets": {
        "h1": "/Volumes/T7/vcc_pp/data/raw/competition_support_set/competition_train.h5"
    },
    "training": {
        "h1": "train"
    },
    "zeroshot": {},
    "fewshot": {}
}

with open("config.toml", "w") as f:
    toml.dump(config, f)

In [15]:
#Prepare the data upfront
import scplode as sp
sdata = sp.read_h5ad("/Volumes/T7/vcc_pp/data/raw/competition_support_set/competition_train.h5")
sdata[0:10]

[INFO] Loading index: obs
[INFO] Loading index: var
[INFO] Loading index: dat (implicitly)


View of AnnData object with n_obs × n_vars = 10 × 18080
    obs: 'target_gene', 'guide_id', 'batch', 'batch_var', 'cell_type'

In [39]:
%%time

import tracemalloc

tracemalloc.start()
start_snapshot = tracemalloc.take_snapshot()

dm = PerturbationDataModule(
    toml_config_path="config.toml",
    embed_key=None,
    num_workers=0,
    batch_col="batch",
    cell_sentence_len=16,
    pert_col="target_gene",
    cell_type_key="cell_type",
    control_pert="non-targeting",
    use_scplode = True,
    output_space = "all"
)
dm.setup()

# # Get training data
train_loader = dm.train_dataloader()

for i, batch in enumerate(train_loader):
    break  # Just test the first batch

end_snapshot = tracemalloc.take_snapshot()
top_stats = end_snapshot.compare_to(start_snapshot, 'lineno')

print(f"Memory used: {sum(stat.size_diff for stat in top_stats) / 1024 / 1024:.2f} MB")
tracemalloc.stop()

/Volumes/T7/vcc_pp/data/raw/competition_support_set/competition_train.h5


Processing h1:   0%|                                                                                         | 0/1 [00:00<?, ?it/s][INFO] Loading index: obs
[INFO] Loading index: var
[INFO] Loading index: dat (implicitly)
Processing h1: 100%|█████████████████████████████████████████████████████████████████████████████████| 1/1 [00:00<00:00,  1.82it/s]


Processed competition_train: 221273 train, 0 val, 0 test
Memory used: 59.39 MB
CPU times: user 7.68 s, sys: 823 ms, total: 8.5 s
Wall time: 9.4 s


In [40]:
%%time

import tracemalloc

tracemalloc.start()
start_snapshot = tracemalloc.take_snapshot()

dm = PerturbationDataModule(
    toml_config_path="config.toml",
    embed_key=None,
    num_workers=0,
    batch_col="batch",
    cell_sentence_len=16,
    pert_col="target_gene",
    cell_type_key="cell_type",
    control_pert="non-targeting",
    use_scplode = False,
    output_space = "all"
)
dm.setup()

# # Get training data
train_loader = dm.train_dataloader()

for i, batch in enumerate(train_loader):
    break  # Just test the first batch

end_snapshot = tracemalloc.take_snapshot()
top_stats = end_snapshot.compare_to(start_snapshot, 'lineno')

print(f"Memory used: {sum(stat.size_diff for stat in top_stats) / 1024 / 1024:.2f} MB")
tracemalloc.stop()

/Volumes/T7/vcc_pp/data/raw/competition_support_set/competition_train.h5


Processing h1: 100%|█████████████████████████████████████████████████████████████████████████████████| 1/1 [00:00<00:00,  3.15it/s]


Processed competition_train: 221273 train, 0 val, 0 test
Memory used: 31.42 MB
CPU times: user 16.2 s, sys: 9.4 s, total: 25.6 s
Wall time: 28 s
